<a href="https://colab.research.google.com/github/satani99/practical_deep_learning_for_coders/blob/main/fast_ai_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

In [3]:
class SiameseImage(Tuple):
  def show(self, ctx=None, **kwargs):
    img1, img2, same_breed = self
    if not isinstance(img1, Tensor):
      if im2.size != img1.size: img2 = img2.resize(img1.size)
      t1, t2 = tensor(img1), tensor(img2)
      t1, t2 = t1.permute(2, 0, 1), t2.permute(2, 0, 1)
    else: t1, t2 = img1, img2
    line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
    return show_image(torch.cat([t1, line, t2], dim=2),
                      title=same_breed, ctx=ctx)

In [9]:
def label_func(fname):
  return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

In [10]:
class SiameseTransform(Transform):
  def __init__(self, fiels, label_func, splits):
    self.labels = files.map(label_func).unique()
    self.lbl2files = {l: L(f for f in files if label_func(f) == l)
                      for l in self.labels}
    self.label_func = label_func
    self.valid = {f: self._draw(f) for f in files[splits[1]]}

  def encodes(self, f):
    f2, t = self.valid.get(f, self._draw(f))
    img1, img2 = PILImage.create(f), PILImage.create(f2)
    return SiameseImage(img1, img2, t)

  def _draw(self, f):
    same = random.random() < 0.5
    cls = self.label_func(f)
    if not same:
      cls = random.choice(L(l for l in self.labels if l != cls))
    return random.choice(self.lbl2files[cls]), same

In [11]:
splits = RandomSplitter()(files)
tfm = SiameseTransform(files, label_func, splits)
tfm(files[0]).show();

TypeError: ignored